In [1]:
# prompt: 구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

HOME = "/content/drive/MyDrive/GroundingDINO"
print(HOME)

/content/drive/MyDrive/GroundingDINO


In [3]:
if not os.path.exists(HOME):
    os.makedirs(HOME)

%cd {HOME}

# GroundingDINO 디렉토리가 존재하지 않을 경우에만 git clone 실행
if not os.path.exists(os.path.join(HOME, "GroundingDINO")):
    !git clone https://github.com/IDEA-Research/GroundingDINO.git

%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow

/content/drive/MyDrive/GroundingDINO
/content/drive/MyDrive/GroundingDINO/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.7 MB/s eta 0:00:00


In [4]:

import os

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

%cd {HOME}

# weights 디렉토리가 존재하지 않을 경우에만 생성
if not os.path.exists(os.path.join(HOME, "weights")):
    os.makedirs(os.path.join(HOME, "weights"))

%cd {HOME}/weights
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

import os

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate

model = load_model(CONFIG_PATH, WEIGHTS_PATH)

/content/drive/MyDrive/GroundingDINO/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True
/content/drive/MyDrive/GroundingDINO
/content/drive/MyDrive/GroundingDINO/weights
/content/drive/MyDrive/GroundingDINO/weights/groundingdino_swint_ogc.pth ; exist: True
/content/drive/MyDrive/GroundingDINO/GroundingDINO


final text_encoder_type: bert-base-uncased


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
IMAGE_DIR = "/content/drive/MyDrive/images"  # 예측할 이미지들이 있는 디렉토리 경로 지정
image_files = [os.path.join(IMAGE_DIR, file) for file in os.listdir(IMAGE_DIR) if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png")]

# 결과 저장
output_dir = "/content/drive/MyDrive/groundingdino_output"
os.makedirs(output_dir, exist_ok=True)

In [10]:
import csv
import cv2
from groundingdino.util.inference import annotate

# CSV 파일 생성
csv_path = os.path.join(output_dir, "groundingdino_predictions.csv")
with open(csv_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["image_path", "object_count", "label", "confidence", "x1", "y1", "x2", "y2"])

# 예측 수행
for image_path in image_files:
    image_source, image = load_image(image_path)
    boxes, logits, phrases = predict(
        model=model,
        image=image,
        caption="bird",  # 인식하고자 하는 객체 이름 설정
        box_threshold=0.4,
        text_threshold=0.3,
        device="cuda"  # GPU 사용
    )

    # 결과 이미지 저장
    annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
    output_image_path = os.path.join(output_dir, f"{os.path.basename(image_path)}_output.jpg")
    cv2.imwrite(output_image_path, annotated_frame)

    # 예측 결과  및 CSV 파일에 저장
    with open(csv_path, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        for box, logit, phrase in zip(boxes, logits, phrases):
            x1, y1, x2, y2 = box
            csv_writer.writerow([image_path, len(boxes), phrase, logit, x1, y1, x2, y2])

SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `B

In [11]:
import csv
import cv2
from groundingdino.util.inference import annotate

# 저장 폴더 이름 설정
output_folder_name = "bird_detection_results"

# 결과 저장 디렉토리 생성
output_dir = os.path.join("/content/drive/MyDrive", output_folder_name)
os.makedirs(output_dir, exist_ok=True)

# CSV 파일 생성 및 헤더 작성
csv_path = os.path.join(output_dir, "groundingdino_predictions.csv")
with open(csv_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["image_path", "object_count", "label", "confidence", "x1", "y1", "x2", "y2"])

# 예측 수행
for image_path in image_files:
    image_source, image = load_image(image_path)
    boxes, logits, phrases = predict(
        model=model,
        image=image,
        caption="bird",  # 인식하고자 하는 객체 이름 설정
        box_threshold=0.3,  # 바운딩 박스 임계값 조정 (기본값: 0.3)
        text_threshold=0.25,  # 텍스트 임계값 조정 (기본값: 0.25)
        device="cuda"  # GPU 사용
    )

    # 결과 이미지 저장
    annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
    output_image_path = os.path.join(output_dir, f"{os.path.basename(image_path)}_output.jpg")
    cv2.imwrite(output_image_path, annotated_frame)

    # 예측 결과 , 파일에 저장
    with open(csv_path, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        for box, logit, phrase in zip(boxes, logits, phrases):
            x1, y1, x2, y2 = box
            csv_writer.writerow([image_path, len(boxes), phrase, logit, x1, y1, x2, y2])

SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `B